<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100"/>

# Cours TAL - Laboratoire 5
# Trois méthodes de désambiguïsation lexicale

**Objectif**

L'objectif de ce laboratoire est d'implémenter et de comparer plusieurs méthodes de désambiguïsation lexicale (en anglais, *Word Sense Disambiguation* ou WSD).  Vous utiliserez un corpus avec plusieurs milliers de phrases, chaque phrase contenant une occurrence du mot anglais *interest* annotée avec le sens que ce mot possède dans la phrase respective.  Les trois méthodes sont les suivantes (elles seront détaillées par la suite) :

1. Algorithme de Lesk simplifié.
1. Utilisation de word2vec.
1. Classification supervisée (cours 9) utilisant des traits lexicaux les mots en position -1, -2, ..., et +1, +2, ..., par rapport à *interest*.

Les méthodes (1) et (2) n'utilisent pas l'apprentissage automatique.  Elles fonctionnent selon le même principe : comparer le contexte d'une occurrence de *interest* avec chacune des définitions des sens (*synsets*) et choisir la définition la plus proche du contexte.  L’algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec la calcule comme la similarité de vecteurs.  La méthode (3) vise à classifier les occurrences de *interest*, les sens étant les classes, en utilisant comme traits les mots du contexte (ce sera de l'apprentissage supervisé).

## 0. Analyse des données

Téléchargez le corpus *interest* depuis le [site du Prof. Ted Pedersen](http://www.d.umn.edu/~tpederse/data.html).  Il se trouve en bas de cette page.  Téléchargez l'archive ZIP marquée *original format without POS tags* et extrayez le fichier `interest-original.txt`.  Téléchargez également le fichier `README.int.txt` indiqué à la ligne au-dessus. Veuillez brièvement répondre aux questions suivantes :

1. Quelles sont les URL du fichier ZIP et celle du fichier `README.int.txt` ?
2. Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?  Est-ce qu'om considère le pluriel *interests* aussi ?  Que se passe-t-il si une phrase contient plusieurs occurrences du mot ?

In [ ]:
# 1) 
# URL du fichier ZIP: https://www.d.umn.edu/~tpederse/Data/interest-original.nopos.tar.gz
# URL du fichier README: https://www.d.umn.edu/~tpederse/Data/README.int.txt

# 2)
# Format du fichier interest-original.txt: Chaque phrase est séparée par $$ et chaque mot est séparé par un espace.

# Les sens du mot interest sont annotés en ajoutant un suffixe au mot "interest" selon son sens: exemple: interest_1 pour le sens 1. Les sens vont de 1 à 6.

# Les pluriel du mots sont pris en compte et sont notés de la façon suivante: interests_1 pour le sens 1 au pluriel.

# Quand les phrases contiennent plusieurs occurences du mot interest, elles sont annotées une par une en répétant la phrase. Lorsque le mot interest n'est pas celui annoté, il est noté préfixé d'une étoile (*interest).


3. D'après le fichier `README.int.txt`, quelles sont les définitions des six sens de *interest* annotés dans les données et quelles sont leurs fréquences ? Vous pouvez copier/coller l'extrait de `README` ici.

In [ ]:
# Sense 1 =  361 occurrences (15%) - readiness to give attention
# Sense 2 =   11 occurrences (01%) - quality of causing attention to be given to
# Sense 3 =   66 occurrences (03%) - activity, etc. that one gives attention to
# Sense 4 =  178 occurrences (08%) - advantage, advancement or favor
# Sense 5 =  500 occurrences (21%) - a share in a company or business
# Sense 6 = 1252 occurrences (53%) - money paid for the use of money

4. De quel dictionnaire viennent les sens précédents ? Où peut-on le consulter en ligne ?  Veuillez aligner les définitions du dictionnaire avec les six sens annotés en écrivant par exemple `Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"`.

In [ ]:
# Les sens viennent du dictionnaire suivant: Longman's Dictionary of Contemporary English
# Le dictionnaire est disponible en ligne: https://www.ldoceonline.com/

# Définitions du dictionnaire pour les 6 sens:
# Sense 1 = if you have an interest in something or someone, you want to know or learn more about them
# Sense 2 = a quality or feature of something that attracts your attention or makes you want to know more about it
# Sense 3 = an activity that you enjoy doing or a subject that you enjoy studying
# Sense 4 = the things that bring advantages to someone or something
# Sense 5 = if you have an interest in a particular company or industry, you own shares in it
# Sense 6 = the extra money that you must pay back when you borrow money

5. En consultant [WordNet en ligne](http://wordnetweb.princeton.edu/perl/webwn), trouvez les définitions des synsets  pour le **nom commun** *interest*.  Combien de synsets y a-t-il ?  Veuillez indiquer comme avant la **définition** de chaque synset pour chacun des six sens ci-dessus (au besoin, fusionner ou ignorer des synsets).

In [ ]:
# Il y a 7 synsets pour le nom commun interest:
# 1) interest, involvement (a sense of concern with and curiosity about someone or something) "an interest in music"
# 2) sake, interest (a reason for wanting something done) "for your sake"; "died for the sake of his country"; "in the interest of safety"; "in the common interest"
# 3) interest, interestingness (the power of attracting or holding one's attention (because it is unusual or exciting etc.)) "they said nothing of great interest"; "primary colors can add interest to a room"
# 4) interest (a fixed charge for borrowing money; usually a percentage of the amount borrowed) "how much interest do you pay on your mortgage?"
# 5) interest, stake ((law) a right or legal share of something; a financial involvement with something) "they have interests all over the world"; "a stake in the company's future"
# 6) interest, interest group ((usually plural) a social group whose members control some field of activity and who have common aims) "the iron interests stepped up production"
# 7) pastime, interest, pursuit (a diversion that occupies one's time and thoughts (usually pleasantly)) "sailing is her favorite pastime"; "his main pastime is gambling"; "he counts reading among his interests"; "they criticized the boy for his limited pursuits"

# Synset associé aux 6 sens:
# Sense 1 = Synset 1
# Sense 2 = Synset 3
# Sense 3 = Synset 7
# Sense 4 = Synset 2
# Sense 5 = Synset 5
# Sense 6 = Synset 4

# Le Synset 6 n'est pas représenté par l'un des 6 sens du mot interest, en effet il s'agit plus du mot interests au pluriel.

6. Définissez (manuellement, ou avec quelques lignes de code) une liste nommée `senses1` avec les mots des définitions du README, en supprimant les stopwords (p.ex. les mots < 4 lettres).  Affichez la liste.

In [4]:
import nltk

In [3]:
from random import randrange

In [5]:
defs = [
  "readiness give attention",
  "quality causing attention given",
  "activity gives attention",
  "advantage advancement favor",
  "share company business",
  "money paid"
]

senses1 = [sentence.split() for sentence in defs]

print(senses1)

[['readiness', 'give', 'attention'], ['quality', 'causing', 'attention', 'given'], ['activity', 'gives', 'attention'], ['advantage', 'advancement', 'favor'], ['share', 'company', 'business'], ['money', 'paid']]


7. En combinant les définitions obtenues aux points (3) et (4) ci-dessus, construisez une liste nommée `senses2` avec pour chacun des sens de *interest* une liste de **mots-clés** correspondants.  Vous pouvez concaténer les définitions, puis écrire des instructions en Python pour extraire les mots (uniques).  Respectez l'ordre des sens données par `README`, et à la fin affichez `senses2`.

In [76]:
defs2 = [
  "involvement concern curiosity", # someone something
  "interestingness power attracting holding attention unusual exciting",
  "pastime pursuit diversion occupies time thoughts pleasantly",
  "sake reason wanting common", # something
  "stake law right legal share financial involvement company",
  "fixed charge borrowing money percentage amount borrowed"
]

senses2 = []
for i in range(len(defs)):
  senses2.append(list(set(defs[i].split() + defs2[i].split())))

print(senses2)

[['concern', 'readiness', 'curiosity', 'give', 'attention', 'involvement'], ['causing', 'given', 'exciting', 'interestingness', 'quality', 'attracting', 'power', 'attention', 'holding', 'unusual'], ['pastime', 'pleasantly', 'gives', 'pursuit', 'time', 'activity', 'occupies', 'thoughts', 'attention', 'diversion'], ['common', 'favor', 'sake', 'advantage', 'wanting', 'advancement', 'reason'], ['legal', 'right', 'law', 'stake', 'company', 'financial', 'share', 'involvement', 'business'], ['money', 'borrowed', 'percentage', 'paid', 'amount', 'fixed', 'charge', 'borrowing']]


8. Chargez les données depuis `interest-original.txt` dans une liste appelée `sentences` qui contient pour chaque phrase la liste des mots (sans les séparateurs *$$* et *===...*).  Les phrases sont-elles déjà tokenisées ?  Sinon, faites-le.  À ce stade, ne modifiez pas encore les occurrences annotées *interest(s)\_X*.  Comptez le nombre total de phrases et affichez-en trois au hasard.

In [8]:
sentences = []
TO_REMOVE = ['$$', '=', '.', ',', '\'', '`', '{', '}', '(', ')', '--']

with open("interest-original.txt", 'r', encoding='utf-8') as fd:
    lines = fd.readlines()
    
    for line in lines:
        cleaned = line
        for char in TO_REMOVE:
            cleaned = cleaned.replace(char, "")
        if cleaned == "\n":
            continue
        sentences.append(nltk.tokenize.word_tokenize(cleaned))

print("Il y a {} phrases.\nEn voici 3 au hasard :".format(len(sentences)))
for i in range(3):
  print(sentences[randrange(len(sentences))])

Il y a 2368 phrases.
En voici 3 au hasard :
['neither', 'the', 'congressional', 'leadership', 'nor', 'the', 'administration', 'has', 'any', 'interest_1', 'in', 'holding', 'up', 'revision', 'of', 'the', 'debt', 'ceiling']
['in', 'london', 'shares', 'were', 'moderately', 'lower', 'in', 'thin', 'trading', 'on', 'continued', 'fears', 'that', 'higher', 'interest_6', 'rates', 'are', 'coming', 'a', 'concern', 'that', 'weighed', 'down', 'many', 'other', 'european', 'markets', 'as', 'well']
['on', 'the', 'contrary', 'it', 'is', 'too', 'responsive', 'to', 'an', 'incorrect', 'view', 'of', 'the', 'public', 'interest_4']


## 1. Algorithme de Lesk simplifié

Définissez une fonction `wsd_lesk(senses, sentence)` qui prend deux arguments : une liste de listes de mots-clés (comme `senses1` et `senses2` ci-dessus) et une phrase avec une occurrence annotée de *interest* ou *interests*, et qui retourne l'index du sens le plus probable (entre 1 et 6) selon l'algorithme de Lesk.  Cet algorithme choisit le sens qui a le maximum de mots en commun avec le contexte de *interest*.  Vous pouvez choisir vous-mêmes la taille de ce voisinage (`window_size`).  En cas d'égalité entre deux sens, tirer la réponse au sort.

In [56]:
# Veuillez répondre ici à la question.  
import re

# Voici quelques fonctions auxiliaires et indications pour vous aider :

# Fonction score : pour deux tableaux de chaînes 'window' et 'sense', 
# retourne combien de chaînes de 'window' sont présentes dans 'sense'.
def score_lesk(window, sense):
    result = 0
    for w in window:
        if w in sense:
            result += 1
    return result

# Fonction max_score_index : retourne la position du score maximal dans le tableau de scores
# (en tirant au sort si plusieurs scores ont la valeur maximale)
def max_score_index(scores):
    m = max(scores)
    p = 0
    pp = []
    for s in scores:
        if s == m:
            pp.append(p)
        p += 1
    if len(pp) == 1:
        return pp[0]
    else:
        return pp[randrange(len(pp))]
            
def wsd_lesk(senses, sentence, window_size = 7):
    # retourne le numéro (de 1 à 6) du sens le plus probable de 'interest' dans la phrase selon la méthode de Lesk
    # window_size = 7 : combien de mots regarder autour de 'interest'

    interest_index = [i for i, w in enumerate(sentence) if re.match('^interest[s]?_[1-6]$', w)][0]
    window = sentence[max(0, interest_index - window_size):min(len(sentence), interest_index + window_size)]
    
    return 1 + max_score_index([score_lesk(window, sense) for sense in senses]) # les sens sont de 1 à 6

Définissez maintenant une fonction `evaluate_wsd(fct_name, senses, sentences)` qui prend en paramètre le nom de la méthode de similarité (pour commencer : `wsd_lesk`) ainsi que la liste des mots-clés par sens, et la liste de phrases, et qui retourne le score de la méthode de similarité.  Ce score sera tout simplement le pourcentage de réponses correctes (sens trouvé identique au sens annoté).

In [150]:
def evaluate_wsd(fct_name, senses, sentences, window_size = 7):
  score = 0
  for sentence in sentences:
    for i, w in enumerate(sentence):
      if re.match('^interest[s]?_[1-6]$', w):
        system_sense = int(w[-1])
        result = fct_name(senses, sentence, window_size)
        if system_sense == result:
          score += 1
  return score / len(sentences)

En optimisant la taille de la fenêtre autour de *interest*, quel est le meilleur score de la méthode de Lesk simplifiée ?  Quelle liste de sens conduit à de meilleurs scores, `senses1` ou `senses2` ?

In [127]:
# Seed pour que les résultats soient reproductibles
import random
random.seed(42)

print('Sense1')
for i in range(1, 20):
  print(f'Window size ({i}): {evaluate_wsd(wsd_lesk, senses1, sentences, i)}')

print('\nSense2')
for i in range(1, 20):
  print(f'Window size ({i}): {evaluate_wsd(wsd_lesk, senses2, sentences, i)}')

Sense1
Window size (1): 0.15118243243243243
Window size (2): 0.17483108108108109
Window size (3): 0.18412162162162163
Window size (4): 0.18538851351351351
Window size (5): 0.1866554054054054
Window size (6): 0.2022804054054054
Window size (7): 0.19805743243243243
Window size (8): 0.19721283783783783
Window size (9): 0.19467905405405406
Window size (10): 0.20016891891891891
Window size (11): 0.19383445945945946
Window size (12): 0.20059121621621623
Window size (13): 0.21199324324324326
Window size (14): 0.20903716216216217
Window size (15): 0.20819256756756757
Window size (16): 0.20523648648648649
Window size (17): 0.2204391891891892
Window size (18): 0.20734797297297297
Window size (19): 0.2179054054054054

Sense2
Window size (1): 0.17229729729729729
Window size (2): 0.18032094594594594
Window size (3): 0.18285472972972974
Window size (4): 0.19045608108108109
Window size (5): 0.20270270270270271
Window size (6): 0.21283783783783783
Window size (7): 0.20988175675675674
Window size (8): 

Le meilleur score de la méthode de Lesk simplifiée tourne autour des 23-24%. La liste `senses2` conduit à de meilleurs scores, ce qui était assez intuitif car elle contient plus de mots-clés. Concernant la taille de la fenêtre, les résultats sont assez équivalents mais une taille de plus de 10 semble être un bon compromis.

## 2. Utilisation de word2vec pour la similarité contexte vs. synset

En réutilisant une partie du code de `wsd_lesk`, définissez maintenant une fonction `wsd_word2vec(senses, sentence)` qui choisit le sens en utilisant la similarité **word2vec**.  On vous encourage à chercher dans la [documentation des KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) comment calculer directement la similarité entre deux listes de mots.

Comme `wsd_lesk`, la nouvelle fonction `wsd_word2vec` prend en argument une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus), et une phrase avec une occurrence annotée de *interest* ou *interests*.  La fonction retourne le numéro du sens le plus probable selon la similarité word2vec entre les mots du sens et ceux du voisinage de *interest*. Vous pouvez choisir la taille de ce voisinage (`window_size`).  En cas d'égalité, tirer le sens au sort.

In [79]:
import gensim
from gensim.models import KeyedVectors
path_to_model = "~/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz"
wv_model = gensim.models.KeyedVectors.load_word2vec_format(path_to_model, binary=True)  # C bin format

In [115]:
def wsd_word2vec(senses, sentence, window_size = 7):
    interest_index = [i for i, w in enumerate(sentence) if re.match('^interest[s]?_[1-6]$', w)][0]
    window = sentence[max(0, interest_index - window_size):min(len(sentence), interest_index + window_size)]
    
    similarities = []
    for sense in senses:
        similarity = wv_model.n_similarity(window, sense)
        similarities.append(similarity)

    return 1 + max_score_index(similarities)

Appliquez maintenant la même méthode `evaluate_wsd` avec la fonction `wsd_word2vec` (en cherchant une bonne valeur de la taille de la fenêtre) et affichez le score de la similarité word2vec.  Comment se compare-t-il avec les précédents ?

In [251]:
# Recherche d'une bonne taille de fenêtre:
# print('Sense1')
# for i in range(1, 20):
#   print(f'Window size ({i}): {evaluate_wsd(wsd_word2vec, senses1, sentences, i)}')

# print('\nSense2')
# for i in range(1, 20):
#   print(f'Window size ({i}): {evaluate_wsd(wsd_word2vec, senses2, sentences, i)}')

window_size = 6

print('Sense1')
print(f'Window size ({window_size}): {evaluate_wsd(wsd_word2vec, senses1, sentences, window_size)}')

print('\nSense2')
print(f'Window size ({window_size}): {evaluate_wsd(wsd_word2vec, senses2, sentences, window_size)}')

Sense1
Window size (6): 0.3572635135135135

Sense2
Window size (6): 0.5903716216216216


Les résultats sont bien meilleurs que les précédents (environ 60% de réponses correctes contre les 23% précédents). Les résultats sont toujours meilleurs avec la liste `senses2` qu'avec la liste `senses1` et la différence est encore plus prononcée qu'avec la méthode de Lesk simplifiée (60% vs 40% environ).

## 3. Classification supervisée avec des traits lexicaux
Dans cette partie du labo, vous entraînerez des classifieurs pour prédire le sens d'une occurrence dans une phrase.  Le principal défi sera de transformer chaque phrase en un ensemble de traits, pour créer les données en vue des expériences de classification.

Vous utiliserez le classifieur `NaiveBayesClassifier` fourni par NLTK.  Le mode d'emploi se trouve dans le [Chapitre 6, sections 1.1-1.3](https://www.nltk.org/book/ch06.html) du livre NLTK.  Consultez-le attentivement pour trouver comment formater les données.  (Il existe de nombreux autres classifieurs supervisés, par exemple dans la boîte à outils `scikit-learn`.)

De plus, vous devrez séparer les 2368 occurrences en ensembles d'entraînement et de test.

Pour représenter les traits lexicaux (*features*), vous les coderez comme `mot-2`, `mot-1`, `mot+1`, `mot+2`, etc. (fenêtre de taille `2*window_size` autour de *interest*) et vous leur donnerez les valeurs des mots observés aux emplacements respectifs, ou alors `NONE` si la fenêtre dépasse la limite de la phrase.  Vous ajouterez un trait qui est le mot *interest* lui-même, qui peut être au singulier ou au pluriel.  Pour chaque occurrence de *interest*, vous devez donc générer une représentation formelle avec un dictionnaire Python suivi de l'index du sens :
```
[{'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': 'NONE', 'word0': 'interest'}, 6]
```
L'index du sens servira à l'entraînement, puis elle sera cachée à l'évaluation, et la prédiction du système sera comparée à elle pour dire si elle est correcte ou non.  Vous regrouperez toutes ces entrées dans une liste totale de 2368 éléments appelée `items_with_features_A`.

En partant de la liste des phrases appelée `sentences`(préparée plus haut), veuillez générer ici cette liste, en vous aidant si nécessaire du livre NLTK.

In [282]:
window_size = 2

items_with_features_A = []
for sentence in sentences:
  dico = dict()
  for index_word, w in enumerate(sentence):
    if re.match('^interest[s]?_[1-6]$', w):
      for i in range(1, window_size + 1):
        dico[f'word-{i}'] = sentence[index_word - i] if index_word - i >= 0 else 'NONE'
        dico[f'word+{i}'] = sentence[index_word + i] if index_word + i < len(sentence) else 'NONE'
      dico['word0'] = w[:-2]
      items_with_features_A.append([dico, int(w[-1])])
      dico = dict()

print(len(items_with_features_A))
print(items_with_features_A[151:154])

2368
[[{'word-1': 'investor', 'word+1': 'in', 'word-2': 'NONE', 'word+2': 'stock', 'word0': 'interest'}, 1], [{'word-1': 'western', 'word+1': 'to', 'word-2': 'the', 'word+2': 'see', 'word0': 'interest'}, 4], [{'word-1': 'of', 'word+1': 'because', 'word-2': 'expression', 'word+2': 'under', 'word0': 'interest'}, 1]]


On souhaite maintenant entraîner un classifieur sur une partie des données, et le tester sur une autre.  Typiquement, on peut garder 80% des données pour l'entraînement et utiliser les 20% restants pour l'évaluation.  Veuillez faire cette division séparément pour chaque sens, pour que les deux ensembles contiennent les mêmes proportions de sens que l'ensemble de départ ("stratification"), et enregistrer les deux sous-ensembles de `items_with_features_A` sous les noms respectifs de `iwf_A_train` et `iwf_A_test`.

In [192]:
from random import shuffle

In [303]:
iwf_A_train = []
iwf_A_test  = []

nb_senses = 6
train_ratio = 0.8

for sense in range(1, nb_senses + 1):
  items_with_features_A_sense = [item for item in items_with_features_A if item[1] == sense]
  shuffle(items_with_features_A_sense)
  iwf_A_train.extend(items_with_features_A_sense[:int(len(items_with_features_A_sense) * train_ratio)])
  iwf_A_test.extend(items_with_features_A_sense[int(len(items_with_features_A_sense) * train_ratio):])

shuffle(iwf_A_train)
shuffle(iwf_A_test)

print(len(iwf_A_train), ' ', len(iwf_A_test))
print(iwf_A_test[:2], iwf_A_test[-2:])

1891   477
[[{'word-1': 'short-term', 'word+1': 'rates', 'word-2': 'other', 'word+2': 'he', 'word0': 'interest'}, 6], [{'word-1': 'institutional', 'word+1': 'in', 'word-2': 'of', 'word+2': 'smaller', 'word0': 'interest'}, 1]] [[{'word-1': 'us', 'word+1': 'rates', 'word-2': 'of', 'word+2': 'remained', 'word0': 'interest'}, 6], [{'word-1': 'as', 'word+1': 'on', 'word-2': 'things', 'word+2': 'debt', 'word0': 'interest'}, 6]]


Veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_A_train` et la tester sur `iwf_A_train` (voir la documentation NLTK).  En expérimentant avec différentes largeurs de fenêtres, quel est le meilleur score global que vous obtenez (avec la fonction `accuracy`), et comment se compare-t-il avec les précédents ?  Quels sont les traits les plus informatifs (voir la doc NLTK), et pouvez-vous expliquer cet affichage ?

In [304]:
from nltk.classify import naivebayes 

classifier = nltk.classify.NaiveBayesClassifier.train(iwf_A_train)
print(nltk.classify.accuracy(classifier, iwf_A_test))

0.8867924528301887


En testant avec différentes valeurs de `window_size`, nous nous rendons compte que les variations sont minimes entre une taille de window_size et une autre. Nous avons donc retenu 2 comme taille de fenêtre car les résultats sont quand même meilleurs qu'avec 1 et pas moins bons qu'avec une fenêtre plus grande. Si la fenêtre est plus grande que 3, les résultats semblent être légèrement moins bons.

Le meilleur score globale tourne atour de **88-89%** de précision. Le score varie en fonction du split des données entre train et test (qui est aléatoire) mais reste toujours au-dessus des 80%.

In [305]:
step1 = evaluate_wsd(wsd_lesk, senses2, sentences, 10)
step2 = evaluate_wsd(wsd_word2vec, senses2, sentences, 6)
step3 = nltk.classify.accuracy(classifier, iwf_A_test)

print('Step 1 (Lesk):')
print(step1)
print('Step 2 (word2vec):')
print(step2)
print('Step 3 (supervised classifier):')
print(step3)

Step 1 (Lesk):
0.23226351351351351
Step 2 (word2vec):
0.5903716216216216
Step 3 (supervised classifier):
0.8867924528301887


Nous observons que les résultats du classifieur entraîné sont bien meilleurs que ceux obtenus grâce aux deux techniques précédentes (Lesk simplifié et word2vec). Cela s'explique par le fait que seul le classifieur utilise de l'apprentissage automatique et peut donc apprendre à reconnaître les différents sens du mot *interest* contrairement aux deux premières méthodes qui utilisent une approche plus directe en comparant simplement des listes de mots. De plus, la 3ème méthode est la seule qui réalise un vrai test en testant sur des valeurs qui n'ont pas été utilisées pour l'entraînement. Les deux premières méthodes ne font que calculer la précision sur les données d'entraînement. Ce qui est encore un avantage à la dernière méthode.

In [308]:
classifier.show_most_informative_features(20)

Most Informative Features
                  word+1 = 'in'                5 : 6      =    103.3 : 1.0
                   word0 = 'interests'         3 : 1      =     64.7 : 1.0
                  word-1 = 'other'             3 : 6      =     41.6 : 1.0
                  word+1 = 'of'                4 : 6      =     30.6 : 1.0
                  word+2 = 'and'               6 : 5      =     23.4 : 1.0
                  word-2 = 'have'              1 : 6      =     19.5 : 1.0
                  word-1 = 'own'               4 : 6      =     19.5 : 1.0
                  word+1 = 'because'           2 : 6      =     16.7 : 1.0
                  word-2 = 'NONE'              6 : 4      =     15.5 : 1.0
                  word-1 = 'and'               6 : 5      =     14.9 : 1.0
                  word+2 = 'the'               5 : 3      =     13.3 : 1.0
                  word-2 = 'other'             3 : 6      =     13.0 : 1.0
                  word-2 = 'company'           5 : 6      =     12.8 : 1.0

Les traits les plus informatifs sont les suivants:
* savoir si interest est au pluriel

TODO: ici

<!-- 
S'il est plutôt au début ou à la fin
l'apparition de certains mots particuliers sont très informatifs pour certaines classes:
foreign, new, rose, u.s., 50, company. co., pursue, strong, board, holding, corp.
 -->

L'affichage liste les positifs contre négatifs. C'est des probabilités de vraissemblance.

## Fin du laboratoire

Merci de nettoyer votre feuille, exécuter une dernière fois toutes les instructions, sauvegarder le résultat, et soumettre le *notebook* sur Cyberlearn.